# One Variable Equations

In [numerical analysis](/math/numerical_analysis/numerical%20analysis.md) we will be solving equations of the form $f(x) = 0$ where $f(x)$ is a function of $x$.

These are the functions that are used to solve one variable equations.

```mermaid
graph LR
    A[a,b] --> B("function(a,b)")
    B --> D{while \n max iterations}
    D -->|False| B
    D -->|True| C[final guess]
```

In [1]:
# autoreload modules when code is run
%load_ext autoreload

In [2]:
import sys  # import system modules
sys.path.append('../..')  # computer_engineering/ directory

In [3]:
from typing import Callable  # import python type hinting
from sympy import Symbol, lambdify, sympify  # import sympy
from scripts.math.util.functions import get_derivative, get_g_function  # Import own modules

## Closed Methods
- Bisection Method
- False Rule -> (secant method)

In [4]:
def bisection_method(f: Callable[[float], float], a: float, b: float, tol=1e-6, max_iter=100) -> float:
    """
    Iteratively bisect the interval [a, b] until the root is found to within the tolerance.

    :param f: function
    :param a: left bound
    :param b: right bound
    :param tol: tolerance
    :param max_iter: maximum iteration
    :return: root
    """
    if f(a) * f(b) > 0: raise ValueError(f"f(a)^f(b) must have different signs")
    c = (a + b) / 2  # (just for avoiding the error: UnboundLocalError: local variable 'c' referenced before assignment)
    for _ in range(max_iter):
        c = (a + b) / 2  # find the midpoint (bisect the interval)
        if f(c) == 0 or (b - a) / 2 < tol: return c
        if f(a) * f(c) < 0: b = c  # if f(a) and f(c) have different signs, then the root is in the interval [a, c]
        else:  a = c  # otherwise, the root is in the interval [c, b]
    return c

In [5]:
def false_rule(f: Callable[[float], float], a:float, b:float, tol=1e-6, max_iter=100) -> float:
# Is really similar to the secant method, but it uses the midpoint of the interval [a, b] instead of the secant line.
    """
    Iteratively approximate the root using (a*f(b) - b*f(a)) / (f(b) - f(a)) until the root is found to within the tolerance.
    This method consist in bisecting the interval [a, b] until the root is found to within the tolerance.
    :param f: function
    :param a: left bound
    :param b: right bound
    :param tol: tolerance
    :param max_iter: maximum iteration
    :return: root
    """
    if f(a) * f(b) > 0: raise ValueError(f"f(a)^f(b) must have different signs")
    c = b - ((f(b) * (b - a)) / (f(b) - f(a)))  # (just for avoiding the error: UnboundLocalError: local variable 'c' referenced before assignment)
    for _ in range(max_iter):
        # c = (a*f(b) - b*f(a)) / (f(b) - f(a))  is equal to c = b - ((f(b) * (b - a)) / (f(b) - f(a))) so IT'S THE SAME
        c = b - ((f(b) * (b - a)) / (f(b) - f(a)))  # find the midpoint (approximate the root)
        if f(c) == 0 or (b - a) / 2 < tol: return c
        if f(a) * f(c) < 0: b = c  # if f(a) and f(c) have different signs, then the root is in the interval [a, c]
        else:  a = c  # otherwise, the root is in the interval [c, b]
    return c


## Open Methods
- `Fixed Point Iteration` -> Gauss-Seidel Method
- Secant Method <- False Rule
- Newton-Raphson Method

In [6]:
def secant_method(f: Callable[[float], float], x0: float, x1: float, tol=1e-6, max_iter=100) -> float:
# Is really similar to the false rule method, but is USES THE LAST TWO APPROXIMATIONS TO FIND THE NEXT ONE.
    """
    Iteraitvely evaluate f(x) to find a closer approximation of the root. 
    Use x_new = x - f(x) * (x - x_old) / (f(x) - f(x_old)) to find the next approximation.

    Parameters
    ----
    :param f: function
    :param x0: initial guess
    :param x1: initial guess
    :param tol: tolerance
    :param max_iter: maximum iteration
    :return: root
    """
    x = x0  # initial guess
    xnew = x1  # (just for avoiding the error: UnboundLocalError: local variable 'xnew' referenced before assignment)
    for _ in range(max_iter):
        xnew = x - f(x) * (x - x1) / (f(x) - f(x1))  # next iteration
        if abs(xnew - x) < tol: return xnew  # if the difference between the two iterations is less than the tolerance, then the root is found
        x1 = x  # update the value of x1 (the previous value of x)
        x = xnew  # update the value of x (the current value of x)
    return xnew

In [7]:
def newton_raphson_method(f: Callable[[float], float], x0: float, tol=1e-6, max_iter=100) -> float:
    """
    Iteratively evaluate f(x) and f'(x) to find a closer approximation of the root.
    Use x_new = x - f(x) / f'(x) to find the next approximation.
    :param f: function
    :param x0: initial guess
    :param tol: tolerance
    :param max_iter: maximum iteration
    :return: root
    """
    x = x0  # initial guess
    xnew = x0  # initial guess (just for avoiding the error: UnboundLocalError: local variable 'xnew' referenced before assignment)
    diff = get_derivative(f)
    for _ in range(max_iter):
        xnew = x - f(x) / diff(x)  # next iteration
        if abs(xnew - x) < tol: return float(xnew)  # if the difference between the two iterations is less than the tolerance, then the root is found
        x = xnew  # update the value of x
    return float(xnew)  # return the root in a float format

In [8]:
x = Symbol('x', real=True)
f = x**3 - 2*x - 5
# f = x**2 - 2
# f = x**2 + 4
f = lambdify(x, f, 'numpy')  # Convert the sympy expression to a numpy function -> f(x) = x**3 - 2*x - 5

a, b = 1, 4  # interval

# ? open methods
print(f'Bisection method: {bisection_method(f, a, b)}')
print(f'False rule method: {false_rule(f, a, b)}')
# ? closed methods
print(f'Secant method: {secant_method(f, a, b)}')   
print(f'Newton-Raphson method: {newton_raphson_method(f, 2)}')
# ! test
# print(f'Fixed-point method: {fixed_point_method(f, a)}')


Bisection method: 2.094550848007202
False rule method: 2.094551481542326
Secant method: 2.0945514815415383
f(x) = x**3 - 2*x - 5
f'(x) = 3*x**2 - 2
Newton-Raphson method: 2.0945514815423265
